In [6]:
from __future__ import annotations

import os
from dotenv import load_dotenv

import dspy
from datasets import load_dataset
from dspy.teleprompt.gepa.instruction_proposal import MultiModalInstructionProposer as DefaultMultiModalProposer

load_dotenv()

# Disable cache for fresh runs
dspy.configure_cache(enable_disk_cache=False, enable_memory_cache=False)


class ImageCaption(dspy.Signature):
    """Generate a caption for an image that describes the text visible in the image."""
    image: dspy.Image = dspy.InputField(desc="Image containing text that needs to be read and described")
    caption: str = dspy.OutputField(desc="A detailed caption describing the text and context visible in the image")


def load_textcaps_dataset(num_train: int = 20, num_val: int = 10, num_test: int = 10):
    """
    Load TextCaps dataset from HuggingFace.
    
    TextCaps is an image captioning dataset that requires reading text in images.
    """
    print("Loading TextCaps dataset...")
    
    try:
        # Try loading from HuggingFace
        dataset = load_dataset("lmms-lab/TextCaps", split="train")
    except Exception as e:
        print(f"Error loading from HuggingFace: {e}")
        print("Trying alternative dataset name...")
        try:
            dataset = load_dataset("textcaps", split="train")
        except Exception as e2:
            print(f"Error loading alternative: {e2}")
            raise Exception("Failed to load TextCaps dataset. Please check your internet connection and dataset availability.")

    
    # Convert to DSPy format
    train_set = []
    val_set = []
    test_set = []
    
    # Process train split
    processed = 0
    skipped = 0
    
    for i, example in enumerate(dataset.select(range(min(num_train, len(dataset))))):
        try:
            # Try to get image URL first (preferred for dspy.Image)
            image_url = None
            # Try URL fields first
            for key in ["flickr_original_url", "flickr_300k_url", "image_url", "url"]:
                if key in example and example[key] is not None:
                    url_value = example[key]
                    if isinstance(url_value, str) and url_value.strip():
                        image_url = url_value.strip()
                        break
            
            # If no URL, try image path
            image_path = None
            if not image_url:
                for key in ["image_path", "path", "file_name", "filename"]:
                    if key in example and example[key] is not None:
                        path_value = example[key]
                        if isinstance(path_value, str) and path_value.strip():
                            image_path = path_value.strip()
                            break
            
            # If we have a PIL Image but no URL/path, we'll need to skip or convert
            # For now, skip if no URL or path
            if not image_url and not image_path:
                skipped += 1
                if i < 3:  # Print first few for debugging
                    print(f"Example {i}: No image URL or path found. Keys: {list(example.keys())}")
                continue
            
            # Try multiple field name variations for caption
            caption = None
            for key in ["caption_str", "caption", "caption_text", "text", "text_caption", "str", "label", "reference_strs"]:
                if key in example and example[key] is not None:
                    caption_value = example[key]
                    # Handle list of captions (take first one)
                    if isinstance(caption_value, list):
                        if len(caption_value) > 0:
                            caption = str(caption_value[0]).strip()
                    else:
                        caption = str(caption_value).strip()
                    if caption:  # Only use if non-empty
                        break
            
            if not caption:
                skipped += 1
                if i < 3:  # Print first few for debugging
                    print(f"Example {i}: No caption found. Keys: {list(example.keys())}")
                continue
            
            # Create dspy.Image from URL or path
            if image_url:
                img = dspy.Image(url=image_url)
            elif image_path:
                if image_path.startswith("http"):
                    img = dspy.Image(url=image_path)
                else:
                    img = dspy.Image(path=image_path)
            else:
                skipped += 1
                if i < 3:
                    print(f"Example {i}: Could not create image from available data")
                continue
            
            train_set.append(
                dspy.Example(image=img, caption=caption).with_inputs("image")
            )
            processed += 1
            
        except Exception as e:
            skipped += 1
            if i < 3:  # Print first few errors for debugging
                print(f"Error processing example {i}: {e}")
                import traceback
                traceback.print_exc()
            continue
    
    print(f"Loaded {len(train_set)} training examples (processed: {processed}, skipped: {skipped})")
    
    if len(train_set) == 0:
        print("\nDebugging info:")
        if len(dataset) > 0:
            print(f"First example keys: {list(dataset[0].keys())}")
            print(f"First example values types: {[(k, type(v)) for k, v in dataset[0].items()]}")
    
    return train_set, val_set, test_set


def textcaps_metric(gold, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Metric for TextCaps: Compare predicted caption with gold caption.
    Uses simple string matching - can be enhanced with BLEU/ROUGE scores.
    """
    pred_caption = getattr(pred, "caption", "").strip().lower()
    gold_caption = getattr(gold, "caption", "").strip().lower()
    
    # Simple exact match
    if pred_caption == gold_caption:
        score = 1.0
        feedback = "Perfect match!"
    # Check for word overlap
    elif pred_caption and gold_caption:
        pred_words = set(pred_caption.split())
        gold_words = set(gold_caption.split())
        overlap = len(pred_words & gold_words)
        total = len(gold_words)
        score = overlap / total if total > 0 else 0.0
        feedback = f"Word overlap: {overlap}/{total} words. Expected: '{gold.caption}', Got: '{pred_caption}'"
    else:
        score = 0.0
        feedback = f"Missing caption. Expected: '{gold.caption}', Got: '{pred_caption}'"
    
    return dspy.Prediction(score=score, feedback=feedback)


# Configure LM and load dataset
dspy.settings.configure(
    lm=dspy.LM("gpt-5-nano", temperature=1.0, max_tokens=16000, cache=False)
)

trainset, valset, testset = load_textcaps_dataset(num_train=20, num_val=10, num_test=10)

if not trainset:
    raise Exception("No training data available. Please check the dataset loading.")

# Use validation set as train if no separate valset
if not valset:
    valset = trainset[:5]  # Use first 5 as validation
    trainset = trainset[5:]  # Rest as training

print(f"\nDataset: {len(trainset)} train, {len(valset)} val examples")

Loading TextCaps dataset...
Loaded 20 training examples (processed: 20, skipped: 0)

Dataset: 15 train, 5 val examples


In [3]:
# ============================================================================
# GEPA with Default Multimodal Proposer (single LLM, no reranking)
# ============================================================================

print("\n" + "="*70)
print("Running GEPA with Default Multimodal Proposer (single LLM)")
print("="*70)

# Use default multimodal proposer (single LLM, no reranking)
default_proposer = DefaultMultiModalProposer()

program = dspy.Predict(ImageCaption)

optimizer = dspy.GEPA(
    metric=textcaps_metric,
    auto="light",
    candidate_selection_strategy="current_best",
    instruction_proposer=default_proposer,
)

default_program = optimizer.compile(program, trainset=trainset, valset=valset)

# Evaluate on validation set
from dspy.evaluate import Evaluate
evaluate = Evaluate(metric=textcaps_metric, num_threads=1)
default_score = evaluate(default_program, valset=valset)

print(f"\nDefault Multimodal Proposer Final Score: {default_score:.2%}")

# Save results
default_program.save("gepa_textcaps_default.json")
print("Saved to: gepa_textcaps_default.json")


2025/11/16 20:27:39 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 400 metric calls of the program. This amounts to 20.00 full evals on the train+val set.
2025/11/16 20:27:39 INFO dspy.teleprompt.gepa.gepa: Using 5 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.



Running GEPA with Default Multimodal Proposer (single LLM)


GEPA Optimization:   0%|          | 0/400 [00:00<?, ?rollouts/s]2025/11/16 20:27:54 INFO dspy.evaluate.evaluate: Average Metric: 2.0277777777777777 / 5 (40.6%)
2025/11/16 20:27:54 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.40555555555555556
GEPA Optimization:   1%|▏         | 5/400 [00:14<19:22,  2.94s/rollouts]2025/11/16 20:27:54 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.40555555555555556


Average Metric: 1.80 / 3 (60.0%): 100%|██████████| 3/3 [00:12<00:00,  4.09s/it]

2025/11/16 20:28:06 INFO dspy.evaluate.evaluate: Average Metric: 1.8 / 3 (60.0%)


...

2025/11/16 21:17:20 INFO dspy.teleprompt.gepa.gepa: Iteration 59: Proposed new text for self: - Task definition: Generate a concise, factual caption that describes the image’s main subject and any clearly legible text present in the scene.

- Visual analysis requirements:
  - Identify the primary subject (people, object, scene) and the main action or state.
  - Scan for legible text within the image (on clothing, signage, packaging, book/album covers, labels, etc.) and note what it says.
  - Note salient visual cues that help identify the subject (e.g., brand logos, product type, country/nationality cues, event context) without overloading the caption with non-essential details.
  - If text is clear and central to identification, integrate it into the caption; if text is blurred or unreadable, rely on visual cues alone.

- Text integration rules:
  - Include clearly legible text when it helps identify the object or context (e.g., brand names, product names, titles, country names).
  - 

Average Metric: 1.24 / 3 (41.5%): 100%|██████████| 3/3 [00:10<00:00,  3.35s/it]

2025/11/16 21:17:44 INFO dspy.evaluate.evaluate: Average Metric: 1.2444444444444445 / 3 (41.5%)


2025/11/16 21:18:02 INFO dspy.teleprompt.gepa.gepa: Iteration 60: Proposed new text for self: You are given an image (or images). Produce a concise, natural-language caption that:
- Describes the main scene: the primary objects, their colors, shapes, and spatial relationships (e.g., left of, in front of, on a table, in a restaurant, etc.).
- Includes any clearly legible text only when it helps identify the object or scene (for example, naming the brand or product). Do not transcribe long strings of text or overly reproduce labels.
- Uses simple present tense and avoids unnecessary detail or background clutter. Keep the caption brief (typically one or two phrases or a short sentence).
- Prioritizes visual accuracy over inferred or speculative details. If text is unclear or not essential to identify the scene, omit it or generalize (e.g., "a bottle with a red label" rather than guessing the exact wording).
- Handles single or multiple items by describing the focal item first and then men

TypeError: Evaluate.__init__() missing 1 required keyword-only argument: 'devset'

In [ ]:
from dspy.evaluate import Evaluate
evaluate = Evaluate(metric=textcaps_metric, devset=valset, num_threads=1)
default_score = evaluate(default_program)

from dspy.evaluate import Evaluate

# Evaluate on validation set
evaluate = Evaluate(metric=textcaps_metric, devset=valset, num_threads=1)

gepa_result = evaluate(default_program)
default_score = gepa_result.score

print(f"\nMulti-LLM Proposer Final Score: {multi_llm_score:.2%}")

# Save results
multi_llm_program.save("gepa_textcaps_multi_llm.json")
print("Saved to: gepa_textcaps_multi_llm.json")



2025/11/16 21:19:32 INFO dspy.evaluate.evaluate: Average Metric: 2.0277777777777777 / 5 (40.6%)


In [14]:
# ============================================================================
# GEPA with Multi-LLM Proposer (with reranking)
# ============================================================================

from multi_modal_instruction_proposer import MultiModalInstructionProposer

print("\n" + "="*70)
print("Running GEPA with Multi-LLM Proposer (with reranking)")
print("="*70)

# Configure multi-LLM proposer with different models for diverse proposals
# Using different models for proposal generation, judging, and merging
multi_llm_proposer = MultiModalInstructionProposer(
    proposal_lms=[
        dspy.LM("openai/gpt-4o", temperature=1.0, max_tokens=16000),  # Proposal 1: GPT-4o (conservative, precise)
        dspy.LM("openrouter/anthropic/claude-sonnet-4.5", temperature=1.0, max_tokens=16000), 
        dspy.LM("openrouter/google/gemini-2.5-flash", temperature=0.6, max_tokens=16000),
    ],
    judge_lm=dspy.LM("openrouter/anthropic/claude-sonnet-4.5", temperature=1.0, max_tokens=16000), 
    merger_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000), 
    top_n=2,  # Select top 2 proposals to merge
    verbose=True
)

program = dspy.Predict(ImageCaption)

optimizer = dspy.GEPA(
    metric=textcaps_metric,
    max_metric_calls=25,
    candidate_selection_strategy="current_best",
    instruction_proposer=multi_llm_proposer,
)

multi_llm_program = optimizer.compile(program, trainset=trainset, valset=valset)


2025/11/16 23:36:27 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 25 metric calls of the program. This amounts to 1.25 full evals on the train+val set.
2025/11/16 23:36:27 INFO dspy.teleprompt.gepa.gepa: Using 5 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.



Running GEPA with Multi-LLM Proposer (with reranking)


GEPA Optimization:   0%|          | 0/25 [00:00<?, ?rollouts/s]2025/11/16 23:36:43 INFO dspy.evaluate.evaluate: Average Metric: 1.6527777777777777 / 5 (33.1%)
2025/11/16 23:36:43 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.33055555555555555
GEPA Optimization:  20%|██        | 5/25 [00:16<01:04,  3.24s/rollouts]2025/11/16 23:36:43 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.33055555555555555


Average Metric: 1.74 / 3 (58.1%): 100%|██████████| 3/3 [00:12<00:00,  4.20s/it]

2025/11/16 23:36:56 INFO dspy.evaluate.evaluate: Average Metric: 1.7444444444444445 / 3 (58.1%)


...

2025/11/16 23:47:10 INFO dspy.evaluate.evaluate: Average Metric: 1.1616161616161615 / 3 (38.7%)




Processing component: self

Generating 3 proposals in parallel...
  [Proposal 3] Generated with openrouter/google/gemini-2.5-flash
  [Proposal 2] Generated with openrouter/anthropic/claude-sonnet-4.5
  [Proposal 1] Generated with openai/gpt-4o

Scoring 3 proposals with judge LLM...
  [Proposal 1] Score: 50.0/100 (Dataset: 22.0, Quality: 28.0)
  [Proposal 2] Score: 74.0/100 (Dataset: 36.0, Quality: 38.0)
  [Proposal 3] Score: 40.0/100 (Dataset: 12.0, Quality: 28.0)

Selected top 2 proposals for merging:
  1. Score: 74.0/100
  2. Score: 50.0/100

Merging top 2 proposals...


2025/11/16 23:50:11 INFO dspy.teleprompt.gepa.gepa: Iteration 4: Proposed new text for self: Write one short caption that conveys the main takeaway implied by the image’s visible text.

Output constraints:
- One sentence, ideally 8–14 words.
- No quotes unless the exact wording is the message.
- Do not list multiple text fragments, colors, fonts, or minor scene details.

How to decide what to say:
1) Find the single most important idea the visible text communicates in context.
2) Paraphrase that idea in plain words; avoid verbatim transcription and exhaustive label content.
3) Mention brands only if they are the subject of the image; otherwise generalize (e.g., “airline sponsor”).
4) If quantity is the point, give an approximate count (“about twenty books”).
5) Include numbers only when they carry meaning (e.g., a match score); omit decorative or repeated text.
6) If multiple texts appear, choose the one with highest salience (central, large, or core to the scene’s purpose) and summari

  Merged instruction created (1395 chars)
  Rationale: 1) Unique elements taken and why:
- From Proposal 1: Core emphasis on concise, high-level, conceptual captions; explicit length target; examples that transform literal text into a takeaway (e.g., spon...

[Final] New instruction for self:
  Write one short caption that conveys the main takeaway implied by the image’s visible text.

Output constraints:
- One sentence, ideally 8–14 words.
- No quotes unless the exact wording is the message...


2025/11/16 23:50:20 INFO dspy.evaluate.evaluate: Average Metric: 0.792929292929293 / 3 (26.4%)
2025/11/16 23:50:20 INFO dspy.teleprompt.gepa.gepa: Iteration 4: New subsample score 0.7929292929292929 is not better than old score 1.1616161616161615, skipping
GEPA Optimization:  92%|█████████▏| 23/25 [13:53<01:12, 36.24s/rollouts]


In [16]:
from dspy.evaluate import Evaluate

# Evaluate on validation set
evaluate = Evaluate(metric=textcaps_metric,devset=valset, num_threads=1)

multi_llm_result = evaluate(multi_llm_program)
multi_llm_score = multi_llm_result.score


# Save results
multi_llm_program.save("gepa_textcaps_multi_llm.json")
print("Saved to: gepa_textcaps_multi_llm.json")



2025/11/16 23:52:39 INFO dspy.evaluate.evaluate: Average Metric: 2.0277777777777777 / 5 (40.6%)


Saved to: gepa_textcaps_multi_llm.json


In [ ]:
# ============================================================================
# Comparison
# ============================================================================

print(f"\n{'='*70}")
print("COMPARISON RESULTS")
print(f"{'='*70}")
print(f"Default Multimodal Proposer Score:  {default_score:.2%}")
print(f"Multi-LLM Proposer Score:          {multi_llm_score:.2%}")
print(f"Difference:                         {multi_llm_score - default_score:+.2%}")

if multi_llm_score > default_score:
    print(f"✓ Multi-LLM Proposer is BETTER by {multi_llm_score - default_score:.2%}")
elif default_score > multi_llm_score:
    print(f"✓ Default Proposer is BETTER by {default_score - multi_llm_score:.2%}")
else:
    print("Both proposers performed equally")
